<h1>Warning: You may need <b><u>MY</u></b> modified dataset to run this code</h1>

# Notebook By Alain: Deep learning techniques for MRI brain tumors classification
<p>The Aim of this notebook is to show the progression of my work. What I am doing here is I am trying to explain my pre-processing and modelling.</p>
<p>I began by downloading the prescribed dataset. The dataset contains four classes: no tumor. puitary, glioma, menigngioma. These images have different sizes, so the first part of the preprocessing is to harmonize the dataset by given to all the images the same size. In the data set, the following propotion is respected for training and testing respectively: </p>
<ul>
    <li>no tumor: 395/105</li>
    <li>pituitary: 827/74</li>
    <li>menigngioama: 822/115</li>
    <li>glioma: 826/100</li>
</ul> 
<p>What does the above proportion tell us? It tells us that there are 2475 images for tumorous brain and 395 for non-tumorous brain.</p>
<p> Recall that the modelling part of the work is in two steps, the first consist of building a model which will be able to differentiate between tumorous and non tumorous brain, the secong step consist in a model which is able to differentiate between the type of tumors. If we want to proceed with the first model, we need a dataset in which the images for each label are almost of equal size so that the model shuld not be bias. In this particullar case, i used rotation and fflipping in order to augment the number of images for non-tumorous. I ended with the proportion 2370/2475 for non-tumorous/tumorous, which is more fair. Fpor the second part of the modelling, the images are almost equal in number for different label; hence we don;t need any data augmentation. </p>
<p> Let's now follow the notebook for explanation on preprocessing and modelling</p>

# Setting the database

<p>Here, we are trying to build a database of the images and their labels. Our aim is to build a two stage neural network. The first stage will tell you if the brain is sick or not and the second stage will identify the disease.</p>

In [1]:
#We import some library in this cell and then we are going to use the differents folder to buils the dataset
# A training and testing dataset
from PIL import Image
from numpy import array
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
import warnings

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import optimizers
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dropout,GlobalAveragePooling2D,GlobalMaxPooling2D,Activation,Lambda,ZeroPadding2D
tf.keras.backend.clear_session()
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard

from keras.utils import np_utils
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

import cv2
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
import io
import ipywidgets as widgets
from IPython.display import display, clear_output

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

%matplotlib inline
warnings.simplefilter('ignore')

2023-05-11 08:04:54.822888: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 08:04:55.001767: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-11 08:04:55.001789: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-11 08:04:55.779466: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

### Data Preparation for te first model

<p>In what follows, I transform the images into matrices and I am storing them into a pandas dataframe. I proceed in two steps. The first step consist of building tow datasets for the first model ie one for training and the other for testing the sameting is done for the second model. The pandas dataframe contains two column; one for images, one for labels. For the fist model, we have two label(1: tumorous, 1:non-tumorous). This is what I am doing in the following cells.</p>

In [2]:
labels=['no_tumor','tumor']

In [4]:
X_train = []
y_train = []
image_size = 128
for i in labels:
    folderPath = os.path.join('/home/dsam/AIMS_CMR/Essay_Alain/dataset/Get_to_work/Brain_2_work','Training',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        y_train.append(i)
for i in labels:
    folderPath = os.path.join('/home/dsam/AIMS_CMR/Essay_Alain/dataset/Get_to_work/Brain_2_work','Testing',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        y_train.append(i)
X_train = np.array(X_train)
y_train = np.array(y_train)

100%|██████████| 289/289 [00:00<00:00, 381.21it/s]


In [5]:
X_train, y_train = shuffle(X_train,y_train,random_state=101)

In [6]:
X_train.shape

(4764, 128, 128, 3)

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X_train,y_train,test_size=0.2)

In [8]:
y_train_new=[]
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)

y_test_new=[]
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

## Defining the first model CNN

In [9]:
def ConvBlock(model, layers, filters):
    '''Create [layers] layers consisting of zero padding, a convolution with [filters] 3x3 filters and batch normalization. Perform max pooling after the last layer.'''
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
        model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D((2, 2)))

In [10]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = keras.Sequential()

    # Input image: 75x75x3
    model.add(Lambda(lambda x: x, input_shape=(128, 128, 3)))
    ConvBlock(model, 1, 32)
    # 37x37x32
    ConvBlock(model, 1, 64)
    # 18x18x64
    ConvBlock(model, 1, 128)
    # 9x9x128
    ConvBlock(model, 1, 256)
    # 4x4x128
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(2, (1, 1), activation='relu'))
    model.add(GlobalAveragePooling2D())
    # 4x4x2
    model.add(Activation('sigmoid'))
    
    return model

## Initialising the model 

In [11]:
model = create_model()

2023-05-11 08:05:06.749133: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/dsam/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2023-05-11 08:05:06.749603: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-11 08:05:06.749655: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dsam-HP-ProBook-640-G1): /proc/driver/nvidia/version does not exist
2023-05-11 08:05:06.750463: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appro

## Compiling te model

In [13]:
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=0.0001) , metrics=['accuracy'])

In [ ]:
model.summary()

## Model Training

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.4, batch_size=10, epochs=50, initial_epoch=0)

## Model monitoring

In [16]:
pred = model.predict(X_test)
pred = np.argmax(pred,axis=1)
y_test_new = np.argmax(y_test,axis=1)

30/30 [==============================] - 8s 251ms/step


In [ ]:
y_test_new

In [ ]:
print(classification_report(y_test_new,pred))

## Preparing dataset for the second model

In [19]:
labels1=['pituitary_tumor','meningioma_tumor','glioma_tumor']

In [94]:
X_train1 = []
y_train1 = []
image_size = 128
for i in labels1:
    folderPath = os.path.join('/home/dsam/AIMS_CMR/Essay_Alain/dataset/Get_to_work/Brain_to_work','Training',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train1.append(img)
        y_train1.append(i)
for i in labels1:
    folderPath = os.path.join('/home/dsam/AIMS_CMR/Essay_Alain/dataset/Get_to_work/Brain_to_work','Testing',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train1.append(img)
        y_train1.append(i)
X_train1 = np.array(X_train1)
y_train1 = np.array(y_train1)

100%|██████████| 100/100 [00:00<00:00, 507.89it/s]


In [95]:
X_train1, y_train1 = shuffle(X_train1,y_train1,random_state=101)

In [96]:
X_train1.shape

(2764, 128, 128, 3)

In [97]:
X_train1,X_test1,y_train1,y_test1=train_test_split(X_train1,y_train1,test_size=0.2)

In [98]:
y_train_new1=[]
for i in y_train1:
    y_train_new1.append(labels1.index(i))
y_train1 = y_train_new1
y_train1 = tf.keras.utils.to_categorical(y_train1)

y_test_new1=[]
for i in y_test1:
    y_test_new1.append(labels1.index(i))
y_test1 = y_test_new1
y_test1 = tf.keras.utils.to_categorical(y_test1)

## Building the CNN for the second model

In [ ]:
dropout_rate=0.1
    
ini_input=keras.Input(shape=(128,128,3),name="image")

x = Conv2D(32, (3, 3), activation='relu',  padding='same')(ini_input)
x = MaxPooling2D((2, 2))(x)
x = Dropout(dropout_rate)(x)
x = BatchNormalization()(x)

x = Conv2D(64, (3, 3), activation='relu',  padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(dropout_rate)(x)
x = BatchNormalization()(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(dropout_rate)(x)
x = BatchNormalization()(x)

x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = Dropout(dropout_rate)(x)
x = BatchNormalization()(x)

x = Conv2D(3, (1, 1))(x)
x = Dropout(dropout_rate)(x)
x = BatchNormalization()(x)
x = GlobalMaxPooling2D()(x)
predictions = Activation('softmax')(x)

In [ ]:
model2=keras.Model(inputs=ini_input,outputs=predictions)

In [139]:
sgd = optimizers.SGD(lr=0.00010,momentum=1, nesterov=True)

In [159]:
model2.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
model2.summary()

## Running the model

In [ ]:
history2 = model2.fit(X_train1, y_train1, validation_split=0.4, batch_size=10, epochs=50, initial_epoch=0)

## Evaluating the model

In [146]:
pred2 = model2.predict(X_test1)
pred2 = np.argmax(pred2,axis=1)
y_test_new1 = np.argmax(y_test1,axis=1)

18/18 [==============================] - 4s 195ms/step


In [ ]:
print(classification_report(y_test_new1,pred2))

## Building dataset for the third model

In [ ]:
labels2=['no_tumor','pituitary_tumor','meningioma_tumor','glioma_tumor']

In [149]:
X_train2 = []
y_train2 = []
image_size = 128
for i in labels2:
    folderPath = os.path.join('/home/dsam/AIMS_CMR/Essay_Alain/dataset/Get_to_work/Brain_t2_work',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train2.append(img)
        y_train2.append(i)
X_train2 = np.array(X_train2)
y_train2 = np.array(y_train2)

100%|██████████| 150/150 [00:00<00:00, 433.93it/s]


In [151]:
X_train2, y_train2 = shuffle(X_train2,y_train2,random_state=101)

In [153]:
y_train_new2=[]
for i in y_train2:
    y_train_new2.append(labels2.index(i))
y_train2 = y_train_new2
y_train2 = tf.keras.utils.to_categorical(y_train2)

## Defining the third model

In [221]:
def Final_model3(Im_set):
    y_pred = np.argmax(model.predict(Im_set.reshape(1,128,128,-1)))
    if y_pred != 0:
        y_pred = np.argmax(model2.predict(Im_set.reshape(1,128,128,-1)))+1
    return y_pred

In [155]:
y_predict =Final_model(X_train2)
y_train_new2 = np.argmax(y_train2,axis=1)

15/15 [==============================] - 3s 188ms/step


In [ ]:
x = np.where(np.argmax(y_train2,axis=1)==0)
x

In [220]:
np.argmax(model.predict(X_train2[x1[1]].reshape(1,128,128,-1)))

1/1 [==============================] - 0s 29ms/step


0

In [198]:
x1 = x[0][:20]

In [293]:
x1 = x[0]
x2 = np.where(np.argmax(y_train2,axis=1)==1)[0]
x3 = np.where(np.argmax(y_train2,axis=1)==2)[0]
x4 = np.where(np.argmax(y_train2,axis=1)==3)[0]

In [ ]:
y_predict2 = []
for i in x1:
    y_predict2.append(Final_model3(X_train2[i]))

In [ ]:
y_predict3 = []
for i in x2:
    y_predict3.append(Final_model3(X_train2[i]))

In [ ]:
y_predict4 = []
for i in x3:
    y_predict4.append(Final_model3(X_train2[i]))

In [ ]:
y_predict5 = []
for i in x4:
    y_predict5.append(Final_model3(X_train2[i]))

## Evaluating the model

In [298]:
len(y_predict2)

200

In [299]:
y_predict2.count(0)

176